In [1]:
# %% (IPYNB - UNA SOLA CELDA) — MEJORADO: menos pérdida, débito/crédito, fecha robusta, prints
import os, re, glob, uuid, tempfile
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any, Optional, Tuple

import pandas as pd
from pypdf import PdfReader, PdfWriter
from pdf2image import convert_from_path
from PIL import Image
import pytesseract

import torch
from transformers import AutoImageProcessor, TableTransformerForObjectDetection

# ===========================
# CONFIG
# ===========================
PDF_DIR = '/Users/andresmauriciotrianareina/Downloads/pdf'
OUTPUT_XLSX = f"extractos_full_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"

PASSWORDS: List[str] = ["1030581154"]
MAX_PAGES = None
DPI = 300
LANG_OCR = "spa+eng"

DEFAULT_BANK = "Banco"
DEFAULT_CURRENCY = "COP"

TABLE_DET_MODEL = "microsoft/table-transformer-detection"

# Más permisivo para no perder tablas
DETECTION_SCORE_THRESHOLD = 0.40   # <-- antes 0.50, ahora más recall
SECONDARY_THRESHOLD = 0.30         # <-- si no detecta nada, baja más

# OCR config
TESS_CONFIG = "--psm 6"

# Debug
DEBUG_TABLES = True
DEBUG_SHOW_SAMPLES = 2   # muestra 2 filas “skipped” por tabla

# ===========================
# HEADERS / SINÓNIMOS
# ===========================
HEADER_SYNONYMS = {
    "date": ["fecha", "date", "fec", "fecha mov", "fecha movimiento"],
    "time": ["hora", "time"],
    # ojo: "amount" general, pero también manejaremos DEBIT/CREDIT aparte
    "amount": ["valor", "value", "amount", "importe", "monto", "total", "transacción", "transaccion"],
    "debit": ["debito", "débito", "cargo", "retiro", "egreso", "salida", "dr"],
    "credit": ["credito", "crédito", "abono", "ingreso", "entrada", "cr"],
    "description": ["descripcion", "descripción", "description", "movimiento", "movimientos", "clase de movimiento", "detalle", "concepto"],
    "reference": ["documento", "doc", "ref", "referencia", "cod", "cod trans", "cód trans", "número", "numero", "no.", "num", "nro", "trans", "codtrans", "aut", "autoriz"],
    "location": ["ciudad", "city", "lugar"],
    "channel": ["oficina", "canal", "oficina/canal", "channel", "sucursal", "app", "cajero"],
    "balance": ["saldo", "balance", "sldo", "saldo disponible", "saldo total"],
}

def _norm(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip().lower())

def _contains_any(text: str, keys: List[str]) -> bool:
    t = _norm(text)
    return any(k in t for k in keys)

# ===========================
# EXCEL: limpiar chars ilegales (NO TRUNCA, solo control chars)
# ===========================
_ILLEGAL_RE = re.compile(r"[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]")

def sanitize_excel_str(x: Any) -> Any:
    if x is None:
        return ""
    if isinstance(x, (int, float)):
        return x
    s = str(x)
    s = s.replace("\u0000", " ")
    s = _ILLEGAL_RE.sub(" ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ===========================
# BANCO + LAST4
# ===========================
def detect_bank_and_last4(text: str) -> Tuple[str, str]:
    t = _norm(text)
    bank = DEFAULT_BANK
    if "banco de bogotá" in t or "banco de bogota" in t: bank = "Banco de Bogotá"
    elif "bancolombia" in t: bank = "Bancolombia"
    elif "davivienda" in t: bank = "Davivienda"
    elif "bbva" in t: bank = "BBVA"
    last4 = ""
    m = re.search(r"(?:cuenta|account|cta|tarjeta|card).{0,80}?(\d{4})\b", text, re.IGNORECASE)
    if m: last4 = m.group(1)
    return bank, last4

# ===========================
# MOVEMENT TYPE
# ===========================
def classify_movement_type(description: str) -> str:
    d = _norm(description)
    rules = [
        ("cdt", ["cdt", "certificado de deposito", "certificado de depósito", "título", "titulo"]),
        ("intereses", ["interes", "interés", "rendimiento", "rendimientos"]),
        ("impuesto", ["gmf", "4x1000", "impuesto", "retencion", "retención", "iva"]),
        ("retiro", ["retiro", "atm", "cajero", "withdrawal"]),
        ("transferencia", ["transferencia", "transf", "pse", "ach", "envio", "envío"]),
        ("pago", ["pago", "cuota", "tarj", "tarjeta", "credito", "crédito", "servicio", "manejo"]),
        ("compra", ["compra", "pos", "datáfono", "datafono", "comercio", "apple.com", "bill", "supermercado", "mercado"]),
        ("abono", ["abono", "consignacion", "consignación", "deposito", "depósito", "ingreso", "recaudo"]),
    ]
    for label, kws in rules:
        if any(k in d for k in kws): return label
    return "pago"

# ===========================
# MONEDA + MONTO ROBUSTO
# ===========================
CURRENCY_HINTS = {
    "USD": ["usd", "us$", "u$s", "dolar", "dólar", "dollars"],
    "COP": ["cop", "peso", "pesos", "col$"],
}

def infer_currency(text: str, default: str = "COP") -> str:
    t = _norm(text)
    if any(k in t for k in CURRENCY_HINTS["USD"]): return "USD"
    if any(k in t for k in CURRENCY_HINTS["COP"]): return "COP"
    return default

def parse_amount_string(raw: str) -> Optional[float]:
    if raw is None:
        return None
    s = str(raw).strip()
    if not s:
        return None

    neg = False
    if re.search(r"(^\s*-\s*)|(\(\s*)", s):
        neg = True

    s2 = re.sub(r"[^\d,.\-()]", "", s)
    s2 = s2.replace("(", "").replace(")", "").replace("-", "")

    if not re.search(r"\d", s2):
        return None

    comma = "," in s2
    dot = "." in s2

    if comma and dot:
        last_comma = s2.rfind(",")
        last_dot = s2.rfind(".")
        if last_dot > last_comma:
            s2 = s2.replace(",", "")
        else:
            s2 = s2.replace(".", "")
            s2 = s2.replace(",", ".")
    elif comma and not dot:
        if re.search(r",\d{1,2}$", s2):
            s2 = s2.replace(",", ".")
        else:
            s2 = s2.replace(",", "")
    elif dot and not comma:
        if not re.search(r"\.\d{1,2}$", s2):
            s2 = s2.replace(".", "")

    try:
        val = float(s2)
        return -val if neg else val
    except:
        return None

def parse_amount_and_currency(amount_cell: str, desc: str, default_currency: str = "COP") -> Tuple[Optional[float], str]:
    cur = infer_currency((amount_cell or "") + " " + (desc or ""), default=default_currency)
    amt = parse_amount_string(amount_cell)
    return amt, cur

# Extra: si no hay amount column clara, busca montos en texto
AMT_FALLBACK_RE = re.compile(r"(?<!\w)(\(?-?\$?\s*\d{1,3}(?:[.,\s]\d{3})*(?:[.,]\d{1,2})?\)?)(?!\w)")

def parse_amount_from_text_fallback(row_text: str) -> Optional[float]:
    cands = AMT_FALLBACK_RE.findall(row_text or "")
    if not cands:
        return None
    # usualmente el último número grande de la línea es el monto del movimiento
    # preferimos el candidato con más dígitos
    cands_sorted = sorted(cands, key=lambda s: (len(re.sub(r"\D", "", s)), (row_text.rfind(s))), reverse=True)
    for c in cands_sorted[:5]:
        v = parse_amount_string(c)
        if v is not None and abs(v) >= 0.01:
            return v
    return None

# ===========================
# FECHAS
# ===========================
MONTHS_ES = {
    "ene":1,"enero":1,"feb":2,"febrero":2,"mar":3,"marzo":3,"abr":4,"abril":4,"may":5,"mayo":5,
    "jun":6,"junio":6,"jul":7,"julio":7,"ago":8,"agosto":8,"sep":9,"sept":9,"septiembre":9,
    "oct":10,"octubre":10,"nov":11,"noviembre":11,"dic":12,"diciembre":12,
}

def infer_statement_year(text: str) -> Optional[int]:
    t = text or ""
    m = re.search(r"(periodo|período|extracto|mes)\D{0,40}((19|20)\d{2})", t, re.IGNORECASE)
    if m:
        return int(m.group(2))
    m = re.search(r"(ene|enero|feb|febrero|mar|marzo|abr|abril|may|mayo|jun|junio|jul|julio|ago|agosto|sep|sept|septiembre|oct|octubre|nov|noviembre|dic|diciembre)\D{0,10}((19|20)\d{2})", t, re.IGNORECASE)
    if m:
        return int(m.group(2))
    years = re.findall(r"\b(19\d{2}|20\d{2})\b", t)
    if years:
        from collections import Counter
        y = Counter(years).most_common(1)[0][0]
        return int(y)
    return None

def normalize_date_str(date_raw: str, statement_year: Optional[int]) -> str:
    if not date_raw:
        return ""
    s = str(date_raw).strip()
    if not s:
        return ""

    s_clean = s.replace("\\", "/").replace("-", "/")
    s_clean = re.sub(r"\s+", " ", s_clean).strip()

    m = re.search(r"\b(\d{1,2})\s*/\s*(\d{1,2})(?:\s*/\s*(\d{2,4}))?\b", s_clean)
    if m:
        d = int(m.group(1))
        mo = int(m.group(2))
        y = m.group(3)
        if y:
            y = int(y)
            if y < 100: y += 2000
        else:
            y = statement_year
        if y:
            try:
                return pd.Timestamp(year=y, month=mo, day=d).date().isoformat()
            except:
                return ""

    m = re.search(r"\b(\d{1,2})\s+(\d{1,2})\b", s_clean)
    if m and not re.search(r"\d{1,2}:\d{2}", s_clean):
        d = int(m.group(1)); mo = int(m.group(2)); y = statement_year
        if y:
            try:
                return pd.Timestamp(year=y, month=mo, day=d).date().isoformat()
            except:
                return ""

    s2 = _norm(s_clean)
    m = re.search(r"\b(\d{1,2})\s*(ene|enero|feb|febrero|mar|marzo|abr|abril|may|mayo|jun|junio|jul|julio|ago|agosto|sep|sept|septiembre|oct|octubre|nov|noviembre|dic|diciembre)\s*((19|20)\d{2})?\b", s2)
    if m:
        d = int(m.group(1))
        mo = MONTHS_ES.get(m.group(2), None)
        y = int(m.group(3)) if m.group(3) else statement_year
        if mo and y:
            try:
                return pd.Timestamp(year=y, month=mo, day=d).date().isoformat()
            except:
                return ""

    return ""

# Fallback: si date cell falla, buscar fecha en la fila completa
DATE_FALLBACK_RE = re.compile(r"\b(\d{1,2}\s*/\s*\d{1,2}(?:\s*/\s*\d{2,4})?)\b")

def date_from_row_fallback(row_txt: str, statement_year: Optional[int]) -> str:
    if not row_txt:
        return ""
    m = DATE_FALLBACK_RE.search(row_txt)
    if m:
        return normalize_date_str(m.group(1), statement_year)
    # caso “24Ene” etc
    m2 = re.search(r"\b(\d{1,2})\s*(ene|enero|feb|febrero|mar|marzo|abr|abril|may|mayo|jun|junio|jul|julio|ago|agosto|sep|sept|septiembre|oct|octubre|nov|noviembre|dic|diciembre)\b", _norm(row_txt))
    if m2:
        return normalize_date_str(m2.group(0), statement_year)
    return ""

# ===========================
# PDF ENCRIPTACIÓN -> RENDER
# ===========================
def decrypt_pdf_to_tempfile(pdf_path: str, passwords: List[str]) -> Tuple[Optional[str], Optional[str], str]:
    try:
        reader = PdfReader(pdf_path)
        if not reader.is_encrypted:
            return pdf_path, None, ""
        used = None
        opened = False
        for pw in passwords:
            try:
                r2 = PdfReader(pdf_path)
                ok = r2.decrypt(pw)
                if ok:
                    reader = r2
                    used = pw
                    opened = True
                    break
            except:
                pass
        if not opened:
            return None, None, "No se pudo abrir el PDF con PASSWORDS."

        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
        tmp_path = tmp.name
        tmp.close()

        writer = PdfWriter()
        for p in reader.pages:
            writer.add_page(p)
        with open(tmp_path, "wb") as f:
            writer.write(f)
        return tmp_path, used, ""
    except Exception as e:
        return None, None, f"decrypt_tempfile error: {type(e).__name__}: {e}"

def render_pdf_pages(pdf_path: str, passwords: List[str], dpi: int, max_pages: Optional[int]) -> Tuple[List[Image.Image], Optional[str], str]:
    try:
        pages = convert_from_path(pdf_path, dpi=dpi)
        if max_pages: pages = pages[:max_pages]
        return pages, None, "plain"
    except:
        pass

    for pw in passwords:
        try:
            pages = convert_from_path(pdf_path, dpi=dpi, userpw=pw)
            if max_pages: pages = pages[:max_pages]
            return pages, pw, "userpw"
        except:
            continue

    tmp_path, used_pw, err = decrypt_pdf_to_tempfile(pdf_path, passwords)
    if err or not tmp_path:
        return [], None, "fail"

    try:
        pages = convert_from_path(tmp_path, dpi=dpi)
        if max_pages: pages = pages[:max_pages]
        return pages, used_pw, "tmp"
    finally:
        try:
            if tmp_path != pdf_path and Path(tmp_path).exists():
                os.remove(tmp_path)
        except:
            pass

# ===========================
# TABLE DETECTOR (CPU)
# ===========================
device = torch.device("cpu")
processor = AutoImageProcessor.from_pretrained(TABLE_DET_MODEL)
model = TableTransformerForObjectDetection.from_pretrained(TABLE_DET_MODEL).to(device)
model.eval()

@torch.no_grad()
def detect_tables(pil_img: Image.Image, score_thr=0.40) -> List[Dict[str, Any]]:
    inputs = processor(images=pil_img, return_tensors="pt")
    outputs = model(**inputs)
    target_sizes = torch.tensor([pil_img.size[::-1]])
    res = processor.post_process_object_detection(outputs, threshold=score_thr, target_sizes=target_sizes)[0]
    out = []
    for score, box in zip(res["scores"], res["boxes"]):
        x0, y0, x1, y1 = [int(v) for v in box.tolist()]
        out.append({"bbox": (x0,y0,x1,y1), "score": float(score)})
    out.sort(key=lambda d: (d["score"], (d["bbox"][2]-d["bbox"][0])*(d["bbox"][3]-d["bbox"][1])), reverse=True)
    return out

def pad_bbox(bbox, w, h, pad=18):
    x0,y0,x1,y1 = bbox
    x0 = max(0, x0-pad); y0 = max(0, y0-pad)
    x1 = min(w, x1+pad); y1 = min(h, y1+pad)
    return (x0,y0,x1,y1)

# ===========================
# OCR STRUCTURED
# ===========================
def ocr_data_df(img: Image.Image) -> pd.DataFrame:
    d = pytesseract.image_to_data(img, lang=LANG_OCR, output_type=pytesseract.Output.DATAFRAME, config=TESS_CONFIG)
    d = d.dropna(subset=["text"])
    d["text"] = d["text"].astype(str)
    d = d[(d["text"].str.strip() != "") & (d["conf"] > 0)]
    return d

def cluster_rows(words: pd.DataFrame) -> pd.DataFrame:
    if words.empty:
        words["row_id"] = []
        return words
    w = words.copy()
    w["y_mid"] = w["top"] + (w["height"]/2.0)
    w = w.sort_values(["y_mid","left"]).reset_index(drop=True)
    h_med = float(w["height"].median()) if len(w) else 10.0
    thr = max(6.0, 0.75*h_med)
    row_ids = []
    cur = 0
    prev_y = None
    for ym in w["y_mid"].tolist():
        if prev_y is None:
            row_ids.append(cur); prev_y = ym; continue
        if abs(ym-prev_y) > thr:
            cur += 1; prev_y = ym
        row_ids.append(cur)
    w["row_id"] = row_ids
    return w

def row_text(w: pd.DataFrame, rid: int) -> str:
    r = w[w["row_id"]==rid].sort_values("left")
    return " ".join(r["text"].tolist()).strip()

def pick_header_row(w: pd.DataFrame) -> Optional[int]:
    if w.empty: return None
    best, best_score = None, -1
    for rid in w["row_id"].unique().tolist()[:140]:
        t = row_text(w, rid)
        score = 0
        if _contains_any(t, HEADER_SYNONYMS["date"]): score += 3
        if _contains_any(t, HEADER_SYNONYMS["description"]): score += 3
        if _contains_any(t, HEADER_SYNONYMS["amount"]): score += 2
        if _contains_any(t, HEADER_SYNONYMS["debit"]): score += 2
        if _contains_any(t, HEADER_SYNONYMS["credit"]): score += 2
        if _contains_any(t, HEADER_SYNONYMS["reference"]): score += 1
        if _contains_any(t, HEADER_SYNONYMS["channel"]): score += 1
        if _contains_any(t, HEADER_SYNONYMS["location"]): score += 1
        if _contains_any(t, HEADER_SYNONYMS["balance"]): score += 1
        if score > best_score:
            best_score = score; best = rid
    # más permisivo: con 4 ya lo aceptamos
    return best if best_score >= 4 else None

def infer_columns_from_header(w: pd.DataFrame, header_rid: int) -> List[Tuple[float,float,str]]:
    hdr = w[w["row_id"]==header_rid].sort_values("left").copy()
    if hdr.empty: return []
    hdr["x0"] = hdr["left"]
    hdr["x1"] = hdr["left"] + hdr["width"]
    xs = hdr[["x0","x1","text"]].values.tolist()

    # gap threshold un poco menor para no “pegar” headers distintos
    groups = []
    cur = [xs[0]]
    for prev, nxt in zip(xs, xs[1:]):
        gap = nxt[0] - prev[1]
        if gap > 18:
            groups.append(cur); cur = [nxt]
        else:
            cur.append(nxt)
    groups.append(cur)

    cols = []
    for g in groups:
        x0 = float(min(v[0] for v in g))
        x1 = float(max(v[1] for v in g))
        txt = " ".join(v[2] for v in g).strip()
        cols.append((x0,x1,txt))

    cols = sorted(cols, key=lambda z: z[0])

    fixed = []
    for i,(x0,x1,txt) in enumerate(cols):
        left = x0 - 10
        right = x1 + 10
        if i>0:
            _, prev_x1, _ = cols[i-1]
            left = (prev_x1 + x0)/2.0
        if i < len(cols)-1:
            next_x0, _, _ = cols[i+1]
            right = (x1 + next_x0)/2.0
        fixed.append((left,right,txt))
    return fixed

def map_header_to_field(htxt: str) -> Optional[str]:
    ht = _norm(htxt)
    if _contains_any(ht, HEADER_SYNONYMS["date"]): return "date"
    if _contains_any(ht, HEADER_SYNONYMS["time"]): return "time"
    if _contains_any(ht, HEADER_SYNONYMS["description"]): return "description"
    if _contains_any(ht, HEADER_SYNONYMS["reference"]): return "reference"
    if _contains_any(ht, HEADER_SYNONYMS["location"]): return "location"
    if _contains_any(ht, HEADER_SYNONYMS["channel"]): return "channel"
    if _contains_any(ht, HEADER_SYNONYMS["balance"]): return "balance"

    # IMPORTANT: DEBIT/CREDIT antes que amount genérico
    if _contains_any(ht, HEADER_SYNONYMS["debit"]): return "debit"
    if _contains_any(ht, HEADER_SYNONYMS["credit"]): return "credit"
    if _contains_any(ht, HEADER_SYNONYMS["amount"]): return "amount"
    return None

def assign_words_to_cols(w: pd.DataFrame, cols: List[Tuple[float,float,str]]) -> pd.DataFrame:
    if w.empty or not cols:
        w["col_id"] = -1
        return w
    ww = w.copy()
    ww["x_mid"] = ww["left"] + (ww["width"]/2.0)
    def col_for_x(x):
        for i,(x0,x1,_) in enumerate(cols):
            if x0 <= x <= x1:
                return i
        return -1
    ww["col_id"] = ww["x_mid"].apply(col_for_x)
    return ww

def parse_table_image_to_transactions(table_img: Image.Image, bank: str, last4: str, statement_year: Optional[int]) -> Tuple[List[Dict[str,Any]], Dict[str,Any]]:
    words = cluster_rows(ocr_data_df(table_img))
    header_rid = pick_header_row(words)
    if header_rid is None:
        return [], {"error": "No header detected"}

    cols = infer_columns_from_header(words, header_rid)

    col_map = {}
    for i,(_,_,htxt) in enumerate(cols):
        f = map_header_to_field(htxt)
        if f and f not in col_map:
            col_map[f] = i

    words2 = assign_words_to_cols(words, cols)

    def cell_text(rid: int, colid: int) -> str:
        c = words2[(words2["row_id"]==rid) & (words2["col_id"]==colid)].sort_values("left")
        return " ".join(c["text"].tolist()).strip()

    txs = []
    prev_tx = None
    row_ids = sorted([rid for rid in words2["row_id"].unique().tolist() if rid > header_rid])

    skipped_no_amount = 0
    continued = 0
    kept = 0
    samples_skipped = []

    header_txt = row_text(words2, header_rid)

    for rid in row_ids:
        # extrae textos por columnas si existen
        date_raw = cell_text(rid, col_map["date"]) if "date" in col_map else ""
        time = cell_text(rid, col_map["time"]) if "time" in col_map else ""
        desc = cell_text(rid, col_map["description"]) if "description" in col_map else row_text(words2, rid)

        # referencia / canal / etc
        ref = cell_text(rid, col_map["reference"]) if "reference" in col_map else ""
        loc = cell_text(rid, col_map["location"]) if "location" in col_map else ""
        chan = cell_text(rid, col_map["channel"]) if "channel" in col_map else ""

        # montos: primero DEBIT/CREDIT, luego AMOUNT, luego fallback en texto
        debit_raw = cell_text(rid, col_map["debit"]) if "debit" in col_map else ""
        credit_raw = cell_text(rid, col_map["credit"]) if "credit" in col_map else ""
        amt_raw = cell_text(rid, col_map["amount"]) if "amount" in col_map else ""

        # fecha normalizada: si no hay date_raw, buscar en la fila completa
        date_iso = normalize_date_str(date_raw, statement_year)
        if not date_iso:
            date_iso = date_from_row_fallback(row_text(words2, rid), statement_year)

        # monto: si hay crédito/débito, úsalo con signo
        amount = None
        currency = infer_currency((debit_raw or "") + " " + (credit_raw or "") + " " + (amt_raw or "") + " " + (desc or ""), default=DEFAULT_CURRENCY)

        deb = parse_amount_string(debit_raw) if debit_raw else None
        cre = parse_amount_string(credit_raw) if credit_raw else None

        if cre is not None and (deb is None or abs(cre) >= 0.01):
            amount = abs(cre)  # crédito positivo
        elif deb is not None and (cre is None or abs(deb) >= 0.01):
            amount = -abs(deb) # débito negativo
        else:
            # monto general
            amount, _cur = parse_amount_and_currency(amt_raw, desc, default_currency=DEFAULT_CURRENCY)
            if amount is None:
                # fallback: buscar monto en texto completo de la fila
                amount = parse_amount_from_text_fallback(row_text(words2, rid))

        row_full_txt = row_text(words2, rid)

        # CONTINUACIÓN: si no hay monto y no hay fecha, pero hay texto -> pega al anterior
        if (not date_iso) and (amount is None) and desc and prev_tx is not None:
            prev_tx["description"] = (prev_tx["description"] + " " + desc).strip()
            if ref and not prev_tx.get("reference"): prev_tx["reference"] = ref
            if loc and not prev_tx.get("location"): prev_tx["location"] = loc
            if chan and not prev_tx.get("channel"): prev_tx["channel"] = chan
            continued += 1
            continue

        # si no hay monto, intenta aún una última vez: por texto completo
        if amount is None:
            amount = parse_amount_from_text_fallback(row_full_txt)

        if amount is None:
            skipped_no_amount += 1
            if len(samples_skipped) < DEBUG_SHOW_SAMPLES:
                samples_skipped.append(row_full_txt)
            continue

        tx = {
            "bank": bank,
            "account_last4": last4,
            "date": date_iso,               # YYYY-MM-DD o ''
            "time": time,
            "amount": amount,
            "currency": currency,
            "movement_type": classify_movement_type(desc),
            "reference": ref,
            "merchant": "",
            "location": loc,
            "channel": chan,
            "description": desc if desc else row_full_txt,
            "id": uuid.uuid4().hex,
        }
        txs.append(tx)
        prev_tx = tx
        kept += 1

    dbg = {
        "header_row_id": header_rid,
        "header_text": header_txt,
        "col_map": col_map,
        "n_cols": len(cols),
        "rows_total_after_header": len(row_ids),
        "kept": kept,
        "continued": continued,
        "skipped_no_amount": skipped_no_amount,
        "skipped_samples": samples_skipped,
    }
    return txs, dbg

def fullpage_fallback_transactions(page_img: Image.Image, bank: str, last4: str, statement_year: Optional[int]) -> Tuple[List[Dict[str,Any]], Dict[str,Any]]:
    txs, dbg = parse_table_image_to_transactions(page_img, bank, last4, statement_year)
    dbg["fullpage_fallback"] = True
    return txs, dbg

# ===========================
# MAIN
# ===========================
def process_pdfs(pdf_dir: str, output_xlsx: str) -> Dict[str, Any]:
    pdf_paths = sorted(glob.glob(str(Path(pdf_dir) / "*.pdf")))
    if not pdf_paths:
        raise FileNotFoundError(f"No PDFs en: {pdf_dir}")

    all_txs = []
    all_dbg = []

    for pdf_path in pdf_paths:
        pdf_name = Path(pdf_path).name

        pages, used_pw, method = render_pdf_pages(pdf_path, PASSWORDS, DPI, MAX_PAGES)
        if not pages:
            print(f"\n📄 {pdf_name} -> ❌ No se pudo renderizar (clave o poppler).")
            continue

        # OCR de primera página para: banco, last4, año del extracto
        first_txt = ""
        try:
            first_txt = pytesseract.image_to_string(pages[0].convert("L"), lang=LANG_OCR, config=TESS_CONFIG)
        except:
            pass

        bank, last4 = detect_bank_and_last4(first_txt)
        statement_year = infer_statement_year(first_txt)

        print(f"\n📄 {pdf_name} | pages={len(pages)} | render={method}{' (pw)' if used_pw else ''} | bank={bank} | last4={last4 or 'NA'} | year={statement_year or 'NA'}")

        total_tables_pdf = 0
        kept_pdf = 0

        for p_idx, page_img in enumerate(pages, start=1):
            dets = []
            try:
                dets = detect_tables(page_img, score_thr=DETECTION_SCORE_THRESHOLD)
                if len(dets) == 0:
                    dets = detect_tables(page_img, score_thr=SECONDARY_THRESHOLD)
            except:
                dets = []

            print(f"  - page {p_idx}: tablas_detectadas={len(dets)} (thr={DETECTION_SCORE_THRESHOLD}/{SECONDARY_THRESHOLD})")
            total_tables_pdf += len(dets)

            page_got_rows = 0
            for t_idx, d in enumerate(dets, start=1):
                x0,y0,x1,y1 = d["bbox"]
                # padding para no cortar encabezados o última columna
                x0,y0,x1,y1 = pad_bbox((x0,y0,x1,y1), page_img.size[0], page_img.size[1], pad=18)
                crop = page_img.crop((x0,y0,x1,y1))

                txs, dbg = parse_table_image_to_transactions(crop, bank, last4, statement_year)

                if DEBUG_TABLES:
                    print(f"    table {t_idx}: score={d['score']:.2f} bbox={(x0,y0,x1,y1)}")
                    if "error" in dbg:
                        print(f"      ❌ {dbg['error']}")
                    else:
                        print(f"      header: {dbg['header_text']}")
                        print(f"      col_map: {dbg['col_map']}")
                        print(f"      rows(after header)={dbg['rows_total_after_header']} kept={dbg['kept']} cont={dbg['continued']} skipped_no_amount={dbg['skipped_no_amount']}")
                        if dbg.get("skipped_samples"):
                            for i,smp in enumerate(dbg["skipped_samples"], 1):
                                print(f"        sample_skipped_{i}: {smp[:180]}")

                if txs:
                    for tx in txs:
                        tx["source_pdf"] = pdf_name
                        tx["source_page"] = p_idx
                        tx["source_table"] = t_idx
                    all_txs.extend(txs)
                    page_got_rows += len(txs)
                    kept_pdf += len(txs)
                    all_dbg.append({"pdf": pdf_name, "page": p_idx, "table": t_idx, **dbg})

            # fallback: si no detectó nada o no sacó filas, intenta página completa
            if (len(dets) == 0) or (page_got_rows == 0):
                fallback_txs, fdbg = fullpage_fallback_transactions(page_img, bank, last4, statement_year)
                if DEBUG_TABLES and "error" not in fdbg:
                    print(f"    fallback full-page: kept={fdbg.get('kept',0)} skipped_no_amount={fdbg.get('skipped_no_amount',0)} col_map={fdbg.get('col_map',{})}")
                if fallback_txs:
                    for tx in fallback_txs:
                        tx["source_pdf"] = pdf_name
                        tx["source_page"] = p_idx
                        tx["source_table"] = 0
                    all_txs.extend(fallback_txs)
                    kept_pdf += len(fallback_txs)
                    all_dbg.append({"pdf": pdf_name, "page": p_idx, "table": 0, **fdbg})
                    print(f"    ✅ fallback full-page: +{len(fallback_txs)} filas")

        print(f"  => total tablas detectadas en PDF: {total_tables_pdf} | filas_kept_pdf={kept_pdf}")

    tx_df = pd.DataFrame(all_txs)

    # ---- LIMPIEZA / REGLAS ----
    if not tx_df.empty:
        tx_df["amount"] = pd.to_numeric(tx_df["amount"], errors="coerce")
        tx_df = tx_df[tx_df["amount"].notna()].copy()

        # fecha final: NO borres filas sin fecha; solo normaliza donde exista
        tx_df["date"] = pd.to_datetime(tx_df["date"], errors="coerce").dt.date.astype(str)
        tx_df.loc[tx_df["date"] == "NaT", "date"] = ""

    # columnas objetivo
    target_cols = ["bank","account_last4","date","time","amount","currency","movement_type","reference","merchant","location","channel","description","id"]
    for c in target_cols:
        if c not in tx_df.columns:
            tx_df[c] = ""

    # ordenar y dejar solo 1 sheet
    tx_df = tx_df[target_cols + ["source_pdf","source_page","source_table"]] if "source_pdf" in tx_df.columns else tx_df[target_cols]

    # limpiar caracteres ilegales excel
    for c in tx_df.columns:
        if tx_df[c].dtype == object:
            tx_df[c] = tx_df[c].apply(sanitize_excel_str)

    with pd.ExcelWriter(output_xlsx, engine="openpyxl") as writer:
        tx_df.to_excel(writer, index=False, sheet_name="movimientos")

    print(f"\n✅ Excel generado: {output_xlsx}")
    return {"pdfs": len(pdf_paths), "rows": int(tx_df.shape[0])}

summary = process_pdfs(PDF_DIR, OUTPUT_XLSX)
print("RESUMEN:", summary)

/Users/andresmauriciotrianareina/miniconda3/envs/PruebaXolit/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
/Users/andresmauriciotrianareina/miniconda3/envs/PruebaXolit/lib/python3.12/site-packages/torch/nn/modules/module.py:2446: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-


📄 Extracto - Diciembre  2025.pdf | pages=7 | render=plain | bank=Banco de Bogotá | last4=0193 | year=2025
  - page 1: tablas_detectadas=1 (thr=0.4/0.3)
    table 1: score=1.00 bbox=(115, 1414, 2455, 3006)
      header: Fecha a Descripción del Movimiento Ciudad Oficina/Canal Documento Valor Saldo
      col_map: {'date': 0, 'description': 2, 'location': 3, 'channel': 4, 'reference': 5, 'amount': 6, 'balance': 7}
      rows(after header)=42 kept=42 cont=0 skipped_no_amount=0
  - page 2: tablas_detectadas=1 (thr=0.4/0.3)
    table 1: score=1.00 bbox=(227, 325, 2501, 3175)
      ❌ No header detected
  - page 3: tablas_detectadas=1 (thr=0.4/0.3)
    table 1: score=1.00 bbox=(226, 326, 2502, 3166)
      ❌ No header detected
  - page 4: tablas_detectadas=1 (thr=0.4/0.3)
    table 1: score=0.99 bbox=(229, 325, 2503, 3173)
      ❌ No header detected
  - page 5: tablas_detectadas=1 (thr=0.4/0.3)
    table 1: score=1.00 bbox=(229, 327, 2504, 3170)
      ❌ No header detected
  - page 6: tablas_det